In [3]:
# The really long way
import skrf as rf
import numpy as npy
from skrf.constants import *

T0 = 290. # Both resistors are set at room temperature for now
kB = K_BOLTZMANN
frequency = rf.Frequency(start=1, stop=10000, npoints=100, unit='MHz')

Ic_ma = 20 # bias current in milliamps
beta0 = 100 # current gain
cpi = 1e-12
cu = 1e-12
rbb = 3
ru = 1e9
rb = 1e9
rc = 1e9
re = 1e9
r0 = 1e9

gm = Ic_ma/26 # transconductance
re = 1/gm
rpi = Beta0*re # input resistance
gpi = 1/rpi
scpi = 1j*frequency.w*cpi
scu = 1j*frequency.w*cu
sug_d = scu - gm

ovec = npy.ones(len(frequency))
zvec = npy.zeros(len(frequency))

# Series resistance containing thermal noise within the BJT
rbb_series_y = rf.network_array([[1/rbb*ovec,  -1/rbb*ovec],
                                 [-1/rbb*ovec,  1/rbb*ovec]])

# The next few networks are combined to create the BJT. 
gpi_shunt_a = rf.network_array([[ovec,     zvec],
                                [gpi*ovec, ovec]])

cpi_shunt_a = rf.network_array([[ovec,     zvec],
                                [scpi*ovec, ovec]])

cug_shfb_a = rf.network_array([[scu/sug_d,     ovec/sug_d],
                               [scu*gm/sug_d,   scu/sug_d]])

# This covariance matrix represents the shot noise within the BJT
cov_bjt_y = 2*kB*T0*gm*rf.network_array([[1/beta0*ovec,  zvec],
                                       [zvec,          ovec]])

a_bjt = npy.matmul(gpi_shunt_a, npy.matmul(cpi_shunt_a, cug_shfb_a))

# Create the series resisance network with attached covariance matrix representing thermal noise
nwrkRbb = rf.Network.from_y(rbb_series_y, frequency=frequency)
nwrkRbb.noise_source('passive')

# Create the BJT model and attach the covariance matrix representing shot noise
nwrkBjt = rf.Network.from_a(a_bjt, frequency=frequency)
shot_noise_cov_y = rf.NetworkNoiseCov(cov_bjt_y, form='y')
nwrkBjt.noise_source(shot_noise_cov_y)


NameError: name 'beta0' is not defined